In [71]:
def field_validate(fname, annotation_type=None):
    def decorator(func):
        def wrapper(self, *args):
   
            all_annotations = getattr(self, "__annotations__", None)
            seeking_attr = getattr(self, fname)
            
            if all_annotations is None:
                if annotation_type is not None:
                    setattr(self, "__annotations__", {})
                    all_annotations = {}
                else:
                    raise Exception("Annotations not found")
                
            if annotation_type is not None:
                if all_annotations.get(fname) is None:
                    self.__annotations__[fname] = annotation_type
                    attr_type = annotation_type
                else:
                    raise Exception("Annotation overload")
            else:
                attr_type = all_annotations.get(fname)
                
            if attr_type is None:
                raise Exception("Field annotation not found")

            for arg in args:
                assert isinstance(arg, attr_type), f"Wrong field type. Type should be {attr_type}, but got {type(arg)} instead"
   
            return func(self, *args)
        return wrapper
    return decorator

In [132]:
from typing import Callable, Iterable
        

class FieldValidator:
    def __init__(self, fieldName: str, annotationType: type = None) -> None:
        self._fieldName: str = fieldName
        self._annotationType: type = annotationType
        
    def _create_annotations(self, cls) -> dict[str, type] | dict:
        
        all_annotations = getattr(cls, "__annotations__", None)
        
        # Если аннотаций в классе вообще не прописано то создаем их
        if all_annotations is None:
            setattr(cls, "__annotations__", {})
    
    def _set_annotation_to_class(self, cls, attrName) -> None:
        # Если в параметрах указана аннотация то смотрим
        if self._annotationType is not None:
            
            # берем все существующие аннотации
            all_annotations = getattr(cls, "__annotations__", None)

            # если аннотации уже существуют
            if all_annotations is not None:
                # и аннотация к атрибуту не прописана в классе то записываем свою
                if all_annotations.get(attrName) is None or (all_annotations.get(attrName) == self._annotationType):
                    cls.__annotations__[attrName] = self._annotationType
                # если существует аннотация к атрибуту и в параметрах тоже передана аннотация то поднимаем ошибку  
                else:
                    raise Exception("Annotation overload")
            # если аннотаций не существует то поднимаем ошибку
            else:
                raise Exception("Annotations not found")
            
        else:
            return
    
    def _get_annotation(self, cls, attrName):
        
        # берем все существующие аннотации
        all_annotations = getattr(cls, "__annotations__", None)
        
        # смотрим если нет аннотации к типу
        if all_annotations.get(attrName) is None:
            # и в параметрах не указана аннотация то поднимаем ошибку
            if self._annotationType is None:
                raise Exception("Field annotation not found")
            # если указано то возвращаем указанную в параметрах аннотацию
            else:
                return self._annotationType
        # если аннотация есть изначально то ее и возвращаем
        else:
            return all_annotations.get(attrName)
    
    def _check_args(self, args: Iterable, attr_type: type):
        for arg in args:
            assert isinstance(arg, attr_type), f"Wrong field type. Type should be {attr_type}, but got {type(arg)} instead"
    
    def __call__(self, func):
        def wrapper(cls, *args):
            
            all_annotations = self._get_all_annotations(cls)

            attr_type = self._get_annotation(cls, self_fieldName)
            self._set_annotation_to_class(cls)

            self._check_args(args, attr_type)
            
            return func(cls, *args)
        return wrapper

In [133]:
from typing import Protocol, runtime_checkable

@runtime_checkable
class SimpleProtocol(Protocol):
    _x: int
        
    def printHello(self):
        ...
        

class ProtocolImpl:
    _x: int
        
    def __init__(self):
        self._x = 10
        
    def printHello(self):
        print("hi")

In [141]:
from types import NoneType

class m:
    
    _x: SimpleProtocol | None
 
    def __init__(self):
        self._x = None
  
    @property
    def X(self):
        return self._x
  
    @X.setter
    @FieldValidator("_x")
    def X(self, newX):
        self._x = newX
  
g = m()
g.X = ProtocolImpl()
print(g.X)

In [156]:
from typing import Union, Any
print(Any == str | None)

False


In [9]:
class A:
    ...
    
class B(A):
    def method1(self):
        ...
    
b = B()    

for k in dir(b):
    print(k,getattr(b, k))

__class__ <class '__main__.B'>
__delattr__ <method-wrapper '__delattr__' of B object at 0x7acb9b18b550>
__dict__ {}
__dir__ <built-in method __dir__ of B object at 0x7acb9b18b550>
__doc__ None
__eq__ <method-wrapper '__eq__' of B object at 0x7acb9b18b550>
__format__ <built-in method __format__ of B object at 0x7acb9b18b550>
__ge__ <method-wrapper '__ge__' of B object at 0x7acb9b18b550>
__getattribute__ <method-wrapper '__getattribute__' of B object at 0x7acb9b18b550>
__gt__ <method-wrapper '__gt__' of B object at 0x7acb9b18b550>
__hash__ <method-wrapper '__hash__' of B object at 0x7acb9b18b550>
__init__ <method-wrapper '__init__' of B object at 0x7acb9b18b550>
__init_subclass__ <built-in method __init_subclass__ of type object at 0x58fe248e1990>
__le__ <method-wrapper '__le__' of B object at 0x7acb9b18b550>
__lt__ <method-wrapper '__lt__' of B object at 0x7acb9b18b550>
__module__ __main__
__ne__ <method-wrapper '__ne__' of B object at 0x7acb9b18b550>
__new__ <built-in method __new__ of